In [2]:
import ipywidgets as widgets
from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, OrdinalScale, Lines, Axis, Figure
from ipyleaflet import WidgetControl
import ee
import os
import ee
import geemap
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from ipyleaflet import projections
from ipyleaflet import WidgetControl
from geemap import geojson_to_ee
from ipyleaflet import basemaps
import geemap
from ipywidgets import Button, HBox, VBox
from ipywidgets import HBox, VBox
from ipywidgets import Layout
from IPython.display import display
from IPython.display import HTML,Javascript, display
from datetime import timedelta, datetime
import os
from bqplot import pyplot as plt
import numpy as np
from ipyleaflet import basemaps
import os
import time
from ipywidgets import widgets, Layout, HBox, HTML
import pandas
import datetime



In [3]:
#Map = geemap.Map(center=[40, 100], zoom=4, add_google_map=True)
Map = geemap.Map(center=(40, 100), zoom=4,  basemap=basemaps.Esri.WorldStreetMap)

In [4]:
# windows path 
admin_shp = './chn_admbnda_adm0_ocha_2020.shp'
# mac path
#admin_shp = '/Users/yreinhardt/desktop/3-semester/Studienprojekt/Studyproject-GEE-master/chn_admbnda_adm0_ocha_2020.shp'
admin_geom = geemap.shp_to_ee(admin_shp)
# drawer control
dc = Map.draw_control
Map.addLayer(admin_geom,  {}, 'Administrative Border')
Map

Map(center=[40, 100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [7]:
""" Functions"""
# clear output from map with reset button
def clear_output(b):
    output_widget.clear_output() 

""" Data Tab"""
# get band from input, band = band.value
def get_band(band):
    
    if (band == 'Sulphur dioxide (SO2)'):
        gas = 'SO2_column_number_density'
        product ='COPERNICUS/S5P/OFFL/L3_SO2'
        vis = so2_vis
    elif (band == 'Nitrogen dioxide (NO2)'):
        gas = 'NO2_column_number_density'
        product ='COPERNICUS/S5P/OFFL/L3_NO2'
        vis = no2_vis
    elif (band == 'Carbon monoxide (CO)'):
        gas = 'CO_column_number_density'
        product ='COPERNICUS/S5P/OFFL/L3_CO'
        vis = co_vis
    elif (band == 'Methane (CH4)'):
        gas = 'CH4_column_volume_mixing_ratio_dry_air'
        product = "COPERNICUS/S5P/OFFL/L3_CH4"
        vis = ch4_vis
    elif (band == 'Absorbing Aerosol Index (AAI)'):
        gas = 'absorbing_aerosol_index'
        product = "COPERNICUS/S5P/OFFL/L3_AER_AI"
        vis = aii_vis
    elif (band == 'Formaldehyde (HCHO)'):
        gas = 'tropospheric_HCHO_column_number_density'
        product = "COPERNICUS/S5P/OFFL/L3_HCHO"
        vis = hcho_vis
    elif (band =='Ozone (O3)'):
        gas = 'O3_column_number_density'
        product = "COPERNICUS/S5P/OFFL/L3_O3"
        vis = o3_vis
    
    return gas, product, vis

# get img collection from input
def get_image_collection(gas, product, vis, geom, start_date, end_date):
    img_collection = ee.ImageCollection(product).select(gas).filterDate(str(start_date),str(end_date)).filterBounds(geom)
    return img_collection



def get_geometry():
    if clip_default == True:
        geom = admin_geom
    elif clip_default == False:
        if len(Map.draw_features) == 0:
            geom = admin_geom
        else:
            geom = ee.FeatureCollection(Map.draw_features)
        
    return geom


clip_default = False
# clip collection to geom
def clip(geom, collection):

    if clip_default == True:
        # get drawn features
        clipped = collection.map(lambda image: image.clipToCollection(geom))
    else:
        clipped = collection.map(lambda image: image.clip(geom))
    
    return clipped


""" Operations Tab """

# dict to store created layer
created_layer = {}


def get_layer_by_name(name):
    # get layer by name using dict keys
    layer_list = created_layer.get(name)
    # collection, vis, geom, gas, bandname, startdate, enddate
    return layer_list[0], layer_list[1], layer_list[2], layer_list[3], layer_list[4], layer_list[5], layer_list[6]

# get timerange
def get_time_range(start, end):
    start_date = start
    end_date = end.strftime("%Y-%m-%d")
    length = 1
    list_of_dates = [start_date.strftime("%Y-%m-%d")]
    
    # length and date for timeseries plot (x-axis)
    while start_date.strftime("%Y-%m-%d") != end_date:
        start_date += timedelta(days=1)
        list_of_dates.append(start_date.strftime("%Y-%m-%d"))
        length +=1
        
    return list_of_dates, length


# aggregate image collection, .value = measure
def aggregate_stack(collection, measure):
    if measure == 'Arithmetic Mean':
        agg_stack = collection.mean()
    elif measure == 'Median':
        agg_stack = collection.median()
    elif measure == 'Minimum':
        agg_stack = collection.min()
    elif measure == 'Maximum':
        agg_stack = collection.max()
        
    return agg_stack



# get difference between to images
def get_difference(first_img, second_img):
    layer1, vis, geom, gas, band, start, end = get_layer_by_name(first_img)
    layer2, vis, geom, gas, band, start, end = get_layer_by_name(second_img)
    
    first = ee.Image(layer1)
    second = ee.Image(layer2)
    
    diff = first.subtract(second)
    diff = diff.divide(first).multiply(100)
    diff = diff.clip(geom)

    
    return diff, diff_vis, geom

# filter non types, preprocessing for timeseries
def filter_non_type(array):
    # filter nontype values, extract float, store in new list
    extract_list= []
    for idx, items in enumerate(array):
        for key, val in items.items():
            if type(val) != type(None):
                extract_list.append(array[idx])

    return extract_list

# get datetime interval
from datetime import date, datetime, timedelta
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

# create user specific timeseries plot
def get_plot_info(layer):
    # get existing layerattributes by name
    collection, vis, geom, gas, band, start_date, end_date = get_layer_by_name(layer)
    
    
    # reduce region (mean)
    def use_reducer(img):
        reduced_val = img.reduceRegion(
          reducer=ee.Reducer.mean(),
          geometry= geom,
          scale=1200,
          maxPixels=1e13
        )
        return img.set('reducer', reduced_val)
    

    # loop through collectiona and use reducer on each img
    reduced_col = collection.map(use_reducer)

    # create array, contains dict with band (key), reduced value (value)
    stats = reduced_col.aggregate_array('reducer').getInfo()
    extracted_list = filter_non_type(stats)
    
    # einfach strings oder ints die auf der x achse stehen
    # x muss anzahl datenpunkte matchen
    # if collection is interval prop == 'true'
    col_prop = collection.first().get('interval')
    col_prop = col_prop.getInfo()
    
    # check if collection is interval based on set prop, if true extract timerange intervall as datetime object for x axis
    if col_prop == 'true':
        time_range, length = get_time_range(start_date, end_date)
        # size of collection
        size = collection.size().getInfo()
        # reconstruct interval, timerange/collection size
        interval = int(size/length)+1
        dates = []
        for result in perdelta(start_date, end_date, timedelta(days=interval)):
            dates.append(result)
        x_data = np.array(dates, dtype='datetime64')
        
        collection_type = True # is interval

    else:
        collection_type = False # is not interval
        # case for not interval
        time_range, length= get_time_range(start_date, end_date)
        x_data = np.array(time_range, dtype='datetime64')
        

        
    
        
    # die werte 
    y = [item.get(gas) for item in extracted_list[:length]]
    y_data = np.array(y)
    return x_data, y_data, band, length, collection_type


def check_col_type(col_type, x_data):
    if col_type == True:
        scale = LinearScale()
        ax_x = Axis(label='Interval timestamps since start', label_offset= '3em', scale=scale, grid_lines='solid', tick_rotate=90, tick_style={'text-anchor': 'start'})
        x_data_ = np.arange(len(x_data))
    elif col_type == False:
        scale = DateScale()
        ax_x = Axis(label='Days', label_offset= '3em', scale=scale, grid_lines='solid',tick_format="%d.%m", tick_rotate=90, tick_style={'text-anchor': 'start'})
        x_data_ = x_data
    return ax_x, scale, x_data_

# void function to create timeseries
def create_timeseries():
            #x_sc = DateScale()#LinearScale()#DateScale() #LinearScale(min=0, max=30) #DateScale(date_format = "%m-%d")#"%m-%d"
            y_sc = LinearScale()
            # check plot for two or one collection
            if timeseries_cb.value == True and compare_timeseries_cb.value == False:
                # get plot information
                x_data, y_data, band, length, collection_type = get_plot_info(layername_first_txt.value)
                
                # adjust collection type based on interval or not
                ax_x, x_sc, x_data = check_col_type(collection_type, x_data)
                
                
                    
                
                # nur bei linear scale
                #x_data =np.arange(len(x_data))
                
                
                
                
                factor = adjust_axis_scale(band)
                factor_string = str(factor)
                
                if len(factor_string) > 5:
                    y_data = [y_data/factor]
                    lable_hint = '(divided by factor {})'.format(factor)
                else: 
                    y_data = [y_data*factor]
                    lable_hint = '(multiplied by factor {})'.format(factor)
                
                data = Lines(x=x_data, y=y_data,
                             scales={'x': x_sc, 'y': y_sc},
                             display_legend=True,
                             labels=[layername_first_txt.value], 
                             default_size='6px')
            elif compare_timeseries_cb.value == True and timeseries_cb.value == False:
                # for two collections
                x_data, y_data, band, length, collection_type = get_plot_info(layername_first_txt.value)
                x_data, y_data_2, band,  length, collection_type = get_plot_info(layername_second_txt.value)
                
                # adjust collection type based on interval or not
                ax_x, x_sc, x_data = check_col_type(collection_type, x_data)
                
                factor = adjust_axis_scale(band)
                # check factor if multiplication or division is needed based on length
                factor_string = str(factor)
                if len(factor_string) > 5:
                    y_data = [y_data/factor,y_data_2/factor]
                    lable_hint = '(divided by factor {})'.format(factor)
                else: 
                    y_data = [y_data*factor,y_data_2*factor]
                    lable_hint = '(multiplied by factor {})'.format(factor)
                    
                data = Lines(x=x_data,
                             y=y_data,
                             scales={'x': x_sc, 'y': y_sc}, 
                             display_legend=True,
                             labels=[layername_first_txt.value,layername_second_txt.value],
                             default_size='8px',
                             marker = 'circle',
                             marker_size= 25)
                

            # defince axis #tick_style={'font-size': 9, 
            
            #ax_x = Axis(label='Days', label_offset= '3em', scale=x_sc, grid_lines='solid',tick_format="%d.%m", tick_rotate=90, tick_style={'text-anchor': 'start'})
            #ax_x = Axis(label='', scale=x_sc, grid_lines='solid' ,tick_rotate=90, tick_format="%m-%d", tick_style={'text-anchor': 'start'})
            
            #linear
            #ax_x = Axis(label='', scale=x_sc, grid_lines='solid', tick_style={'text-anchor': 'start'})

            
            ax_y = Axis(label=band + lable_hint, scale=y_sc, orientation='vertical', label_offset= '3em')#tick_format='0.2f',
            
            return ax_x, ax_y, data, length

            

# adjust axis scale based on band for cleaner plots
def adjust_axis_scale(band):
    if band == 'Sulphur dioxide (SO2)':
        factor = 10000 # multiply
    elif band == 'Nitrogen dioxide (NO2)':
        factor = 10000 # multiply
    elif band == 'Formaldehyde (HCHO)':
        factor = 10000 # multiply
    elif band == 'Carbon monoxide (CO)':
        factor = 100 # multiply
    elif band == 'Absorbing Aerosol Index (AAI)':
        factor = 10 # multiply
    elif band == 'Methane (CH4)':
        factor = 1000000 # divide 
    
    return factor
        

def create_splitview(left, right):
    
    # get laye by name 
    collection_left, vis_left, geom, gas, band, start_date, end_date = get_layer_by_name(left)
    collection_right, vis_right, geom, gas, band, start_date, end_date = get_layer_by_name(right)
    # convert to layer
    left_layer = geemap.ee_tile_layer(collection_left, vis_left, left)
    right_layer = geemap.ee_tile_layer(collection_right, vis_right, right)
    
    return left_layer, right_layer



"""Checkbox logic dont allow unintended commands instead of error handling"""

def change_usability_timeseries(b):
    if timeseries_cb.value == True and compare_timeseries_cb.value == False:
        layername_second_txt.disabled = True
        layername_first_txt.disabled = False
        compare_timeseries_cb.disabled = True
        plot_title_txt.disabled = False
        splitview_cb.disabled = True
        splitview_cb.value = False
    elif compare_timeseries_cb.value == True and timeseries_cb.value == False:
        layername_second_txt.disabled = False
        layername_first_txt.disabled = False
        timeseries_cb.disabled = True
        plot_title_txt.disabled = False
        splitview_cb.disabled = True
        splitview_cb.value = False
    elif compare_timeseries_cb.value == False and timeseries_cb.value == False:
        layername_first_txt.disabled = True
        layername_second_txt.disabled = True
        plot_title_txt.disabled = True
        splitview_cb.disabled = False
        
def change_usability_splitview(b):
    if splitview_cb.value == True:
        left_layer_txt.disabled = False
        right_layer_txt.disabled = False
        compare_timeseries_cb.disabled = True
        timeseries_cb.disabled = True
        compare_timeseries_cb.value = False
        timeseries_cb.value = False
        
    elif splitview_cb.value == False:
        left_layer_txt.disabled = True
        right_layer_txt.disabled = True
        compare_timeseries_cb.disabled = False
        timeseries_cb.disabled = False



def change_usability_spatialstat(b):
    if spatial_stat_cb.value == True:
        stack_search_txt.disabled = False
        stat_dd.disabled = False
        
        diff_col_cb.disabled = True
        first_search_txt.disabled = True
        second_search_txt.disabled = True
        
    elif spatial_stat_cb.value == False:
        stack_search_txt.disabled = False
        stat_dd.disabled = False
        
        diff_col_cb.disabled = False
        first_search_txt.disabled = False
        second_search_txt.disabled = False
        


def change_usability_difference(b):
    if diff_col_cb.value == True:
        stack_search_txt.disabled = True
        stat_dd.disabled = True
        spatial_stat_cb.disabled = True
        
    elif diff_col_cb.value == False:
        stack_search_txt.disabled = False
        stat_dd.disabled = False
        spatial_stat_cb.disabled = False



# store n value from slider to slice data_list from widget input
value_n = [0]
# enable widgets based on slider for number of classes
def enable_classes(n):
    value_n.pop()
    value_n.append(n)
    if n == 0:
        class_val_1.disabled= True
        class_val_2.disabled= True
        class_val_3.disabled= True
        class_val_4.disabled= True
        class_val_5.disabled= True 
        
        class_val_1_1.disabled= True 
        class_val_2_1.disabled= True 
        class_val_3_1.disabled= True 
        class_val_4_1.disabled= True 
        class_val_5_1.disabled= True 


        
    elif n == 1:
        class_val_1.disabled= False
        class_val_2.disabled= True
        class_val_3.disabled= True
        class_val_4.disabled= True
        class_val_5.disabled= True
        
        class_val_1_1.disabled= False 
        class_val_2_1.disabled= True 
        class_val_3_1.disabled= True 
        class_val_4_1.disabled= True 
        class_val_5_1.disabled= True 
        

        
    elif n ==2:
        class_val_1.disabled= False
        class_val_2.disabled= False
        class_val_3.disabled= True
        class_val_4.disabled= True
        class_val_5.disabled= True
        
        class_val_1_1.disabled= False 
        class_val_2_1.disabled= False 
        class_val_3_1.disabled= True 
        class_val_4_1.disabled= True 
        class_val_5_1.disabled= True 


    elif n ==3:
        class_val_1.disabled= False
        class_val_2.disabled= False
        class_val_3.disabled= False
        class_val_4.disabled= True
        class_val_5.disabled= True
        
        class_val_1_1.disabled= False 
        class_val_2_1.disabled= False 
        class_val_3_1.disabled= False 
        class_val_4_1.disabled= True 
        class_val_5_1.disabled= True
        

    elif n ==4:
        class_val_1.disabled= False
        class_val_2.disabled= False
        class_val_3.disabled= False
        class_val_4.disabled= False
        class_val_5.disabled= True
                
        class_val_1_1.disabled= False 
        class_val_2_1.disabled= False 
        class_val_3_1.disabled= False 
        class_val_4_1.disabled= False 
        class_val_5_1.disabled= True

    elif n ==5:
        class_val_1.disabled= False
        class_val_2.disabled= False
        class_val_3.disabled= False
        class_val_4.disabled= False
        class_val_5.disabled= False
        
        class_val_1_1.disabled= False 
        class_val_2_1.disabled= False 
        class_val_3_1.disabled= False 
        class_val_4_1.disabled= False 
        class_val_5_1.disabled= False
        
        class_val_1.disabled= False
        class_val_2.disabled= False
        class_val_3.disabled= False
        class_val_4.disabled= False
        class_val_5.disabled= False
        

    
    return


def reclassify(layer, data_list):
    # get existing layer
    img, vis, geom, gas, band, start_date, end_date = get_layer_by_name(layer)

    # liste logic [first_lw, frist_upper, second_lw, second_upper, ....]
    # equal 2 = one class, eqal 4  = two classes.., class 1 for not in range
    if len(data_list) == 2:
        reclass_img = ee.Image(1)  \
        .where(img.gte(data_list[0]).And(img.lte(data_list[1])), 2)
    elif len(data_list) == 4:
        reclass_img = ee.Image(1) \
        .where(img.gte(data_list[0]).And(img.lte(data_list[1])), 2) \
        .where(img.gte(data_list[2]).And(img.lte(data_list[3])), 3)
    elif len(data_list) == 6:
        reclass_img = ee.Image(1) \
        .where(img.gte(data_list[0]).And(img.lte(data_list[1])), 2) \
        .where(img.gte(data_list[2]).And(img.lte(data_list[3])), 3) \
        .where(img.gte(data_list[4]).And(img.lte(data_list[5])), 4) 
    elif len(data_list) == 8:
        reclass_img = ee.Image(1) \
        .where(img.gte(data_list[0]).And(img.lte(data_list[1])), 2) \
        .where(img.gte(data_list[2]).And(img.lte(data_list[3])), 3) \
        .where(img.gte(data_list[4]).And(img.lte(data_list[5])), 4) \
        .where(img.gte(data_list[6]).And(img.lte(data_list[7])), 5)
    elif len(data_list) == 10:
        reclass_img = ee.Image(1) \
        .where(img.gte(data_list[0]).And(img.lte(data_list[1])), 2) \
        .where(img.gte(data_list[2]).And(img.lte(data_list[3])), 3) \
        .where(img.gte(data_list[4]).And(img.lte(data_list[5])), 4) \
        .where(img.gte(data_list[6]).And(img.lte(data_list[7])), 5) \
        .where(img.gte(data_list[8]).And(img.lte(data_list[9])), 6)   
        
    return reclass_img, geom


def change_usability_clip_cb(b):
    if clip_feautre_layer_cb.value == False:
        clip_default = False
    elif clip_feautre_layer_cb.value == True:
        clip_default = True
        
# get drawn feature and convert to ee feature collection
def get_drawn_feature():
    feature_collection = ee.FeatureCollection(Map.draw_features)
    return feature_collection
        
def change_download_img(b):
    if download_img_cb.value == True:
        download_col_cb.disabled = True
        download_feature_cb.disabled = True
    elif download_img_cb.value == False and download_feature_cb.value == False and download_col_cb.value == False:
        download_col_cb.disabled = False
        download_feature_cb.disabled = False

def change_download_col(b):
    if download_col_cb.value == True:
        download_img_cb.disabled = True
        download_feature_cb.disabled = True
    elif download_img_cb.value == False and download_feature_cb.value == False and download_col_cb.value == False:
        download_img_cb.disabled = False
        download_feature_cb.disabled = False

            
def change_download_feat(b):
    if download_feature_cb.value == True:
        download_img_cb.disabled = True
        download_col_cb.disabled = True
    elif download_img_cb.value == False:
        download_img_cb.disabled = False
        download_col_cb.disabled = False
        

# void, add colorbar to map based on layer        
def add_colorbar(viz):
    colors = viz['palette']
    vmin = viz['min']
    vmax = viz['max']
    Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax)

      


# void add legend to map depending on user input, legend colors = reclass_vis["palette"]
def add_legend(legend_colors):
    # create keys based on number of classes
    legend_keys = list(range(1, len(legend_colors)+1))
    Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
    
    
    

# create intervall

def create_interval(collection, start, count, interval, units):
    # Create a sequence of numbers, one for each time interval.
    sequence = ee.List.sequence(0, ee.Number(count).subtract(1))
    
    start = start.strftime("%Y-%m-%d")

    originalStartDate = ee.Date(start)

    def create_interval_(i):
        # Get the start date of the current sequence.
        startDate = originalStartDate.advance(ee.Number(interval).multiply(i), units)

        # Get the end date of the current sequence.
        endDate = originalStartDate.advance(
          ee.Number(interval).multiply(ee.Number(i).add(1)), units)
        
        # convert to readable dates
        startDate_read = ee.Date(startDate).format("YYYY-MM-dd")
        enDate_read = ee.Date(endDate).format("YYYY-MM-dd")
        return collection.filterDate(startDate, endDate).mean() \
            .set('system:time_start', startDate) \
            .set('system:time_end',endDate) \
            .set('interval', 'true')

    return ee.ImageCollection(sequence.map(create_interval_))

In [87]:
#Designe interactive widgets

# basic styling, html, css possible 
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_widget_export = widgets.Output(layout={'width': '1px', 'height':'1px'})

output_widget_e = widgets.Output(layout={'width': '200px', 'height':'300px'})

output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)


""" Layout, css"""

from IPython.display import Javascript, display
style = {'description_width': 'initial'}


align_kw = dict(
    _css = (('.widget-label', 'min-width', '40ex'),),
    margin = '0px 0px 5px 12px'
)

description_style = {'description_width': '75px'}
layout = {'width': '400px'}
tab_layout = Layout(width='100%', height='410px')#Layout(width='50%', height='330px')


""" action buttons"""

submit_btn_data = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

submit_btn_operations = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

submit_btn_vis = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

submit_btn_reclass = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

reset_output = widgets.Button(
    description='Clear Output',
    button_style='danger',
    tooltip='Click me',
    style=style)

label = widgets.Label('Upload local files. Supported Data: Shapefile as Zip-File or GeoJSON')


uploader = widgets.FileUpload(
    description='Upload data',
    accept='.zip, .json, .geojson', 
    multiple=False,
    button_style='primary',
    style=style
)

submit = widgets.Button(
    description='Display data',
    button_style='success',
    tooltip='Click me',
    style=style)

# delete local layers from map
reset_output = widgets.Button(
    description='Clear Output',
    button_style='danger',
    tooltip='Click me',
    style=style)

placeholder = widgets.Text(
    value='value',
    placeholder='Type something',
    description='placeholder:',
    disabled=False
)

# add button clear actual outputs


""" Data Tab """
# dropdown = dd, checkbox = cb, date picker = dp, textfield = txt_input, intfield = int_input


# unique lables as headers
band_header = widgets.Label(value="Sentinel-5P Bands")
layername_header = widgets.Label(value="New Layername")
start_date_header = widgets.Label(value="Start Date")
end_date_header = widgets.Label(value="End Date")
intervall_header = widgets.Label(value="Create Intervall")
unit_header = widgets.Label(value="Unit Days")
clip_header = widgets.Label(value="Clip to Geometry")
geometry_header = widgets.Label(value="Geometry")
header = widgets.Label(value="get space")

clip_info_header = widgets.Label(value="* By default all Layers are clipped to Boundary of China")
# placeholder hier inintialisieren nicht bei operations
placeholder_header = widgets.Label(value="")

header.layout.visibility = "hidden"

band_dd = widgets.Dropdown(
    #description='Band:',
    options=['Sulphur dioxide (SO2)',
             'Nitrogen dioxide (NO2)',
             'Carbon monoxide (CO)',
             'Methane (CH4)',
             'Absorbing Aerosol Index (AAI)',
             'Formaldehyde (HCHO)',
             'Ozone (O3)'],
    value='Sulphur dioxide (SO2)',
    **align_kw,
    style=description_style,
    #layout= layout
)

new_name_txt_input = widgets.Text(
    #value='your layer title...',
    placeholder='Your Layer Title',
    #description='Layername:',
    disabled=False,
)


intervall_cb = widgets.Checkbox(
    value=False,
    #description='Intervall',
    disabled=False,
    indent=False,
    **align_kw,
)

intervall_int_input= widgets.IntText(
    #value=7,
    #description= 'Days',
    disabled=False,
    **align_kw
)

start_date_dd = widgets.DatePicker(
    #description='Start Date',
    disabled=False,
     **align_kw,
    style= description_style
)

end_date_dd = widgets.DatePicker(
    #description='End Date',
    disabled=False,
)

clip_feautre_layer_cb = widgets.Checkbox(
    value=False,
    description='Clip to drawn Features',
    disabled=False,
    indent=False,
     **align_kw
)



items = [band_header,
         band_dd,
         layername_header,
         new_name_txt_input,
         start_date_header,
         start_date_dd,
         end_date_header,
         end_date_dd,
         intervall_header,
         intervall_cb,
         unit_header,
         intervall_int_input,
         clip_header,
         clip_feautre_layer_cb,
         submit_btn_data,
         reset_output,
         header,
         placeholder_header,
         clip_info_header]


cmd_box = HBox([items[14], items[15]])
left_box = VBox([items[0], items[1], items[4], items[5], items[8], items[9], items[12], items[13],items[18],items[17],cmd_box])
right_box = VBox([items[2], items[3], items[6], items[7], items[10], items[11], items[17]])

data_widgets =HBox([left_box, right_box])


""" Operations """

stack_header = widgets.Label(value="Select Stack")
stack_search_header = widgets.Label(value="Search Layername")
spatial_stat_header = widgets.Label(value="Spatial Statistics")
stat_measure_header = widgets.Label(value="Statistical Measure")
placeholder_header = widgets.Label(value="")
stack_header = widgets.Label(value="Select Stack")
difference_header = widgets.Label(value="Calculate Difference (%)")
diff_start_first_header = widgets.Label(value="First Image")
diff_start_second_header = widgets.Label(value="Second Image")
diff_end_header = widgets.Label(value="End Date")
info_calc_header = widgets.Label(value="(Select Geometry & Bands on previous Tab)")

stack_search_txt = widgets.Text(placeholder='Layername',disabled=False)

first_search_txt = widgets.Text(placeholder='Layername',disabled=False)

second_search_txt = widgets.Text(placeholder='Layername',disabled=False)

select_staack_cb = widgets.Checkbox(value=False,disabled=False,indent=False,**align_kw)

spatial_stat_cb = widgets.Checkbox(value=False,disabled=False,indent=False,**align_kw,)

stat_dd = widgets.Dropdown(options=['Arithmetic Mean', 'Median', 'Minimum', 'Maximum'],value='Arithmetic Mean', **align_kw,style=style)

diff_col_cb = widgets.Checkbox(value=False,disabled=False,indent=False,**align_kw)

layername_txt_input = widgets.Text(placeholder='Choose Layer by Name',disabled=False,)

new_name_txt_input_op = widgets.Text(placeholder='Your Layer Title',disabled=False,)

# reuse date picker, layername from data tab for difference calculation

items = [stack_header,
         select_staack_cb,
         stack_search_header,
         stack_search_txt,
         spatial_stat_header,
         spatial_stat_cb,
         placeholder_header,
         stat_dd,
         difference_header,
         diff_col_cb,
         diff_start_first_header,
         first_search_txt,
         diff_start_second_header,
         second_search_txt,
         layername_header,
         new_name_txt_input_op,
         stat_measure_header,
         submit_btn_operations,
         info_calc_header,
         reset_output]



cmd_box = HBox([items[17], items[19]])

left_box = VBox([items[4], items[5], items[6], items[6],items[8], items[9], items[14], items[15],items[6],items[6],cmd_box])
right_box = VBox([items[2], items[3], items[16], items[7], items[10], items[11], items[12], items[13]])

operations_widgets =HBox([left_box, right_box])

""" Visualization """

timeseries_header = widgets.Label(value="Timeseries")
splitview_header = widgets.Label(value="Splitview")
splitview_left_header = widgets.Label(value="Left Image")
splitview_right_header = widgets.Label(value="Right Image")
search_second_header = widgets.Label(value="Search 2nd Layername")
plot_tile_header = widgets.Label(value="Plot Title")


timeseries_cb = widgets.Checkbox(value=False,description="Timeseries for one Collection",disabled=False,indent=False,**align_kw,)

compare_timeseries_cb = widgets.Checkbox(value=False,description="Compare two Collections",disabled=False,indent=False,**align_kw)

layername_first_txt = widgets.Text(placeholder='Choose Layer by Name',disabled=True)

layername_second_txt = widgets.Text(placeholder='Choose Layer by Name',disabled=True)

plot_title_txt = widgets.Text(placeholder='Plot Title',disabled=True)

splitview_cb = widgets.Checkbox(value=False,description='Use aggregated Image',disabled=False,indent=False,**align_kw)

left_layer_txt = widgets.Text(placeholder='Choose Layer by Name',disabled=True,)

right_layer_txt = widgets.Text(placeholder='Choose Layer by Name',disabled=True,)

items = [splitview_header,
         splitview_cb,
         splitview_left_header,
         splitview_right_header,
         stack_search_txt,
         timeseries_header,
         timeseries_cb,
         stack_search_header,
         layername_first_txt,
         placeholder_header,
         submit_btn_vis,
         search_second_header,
         compare_timeseries_cb,
         layername_second_txt,
         plot_title_txt,
         plot_tile_header,
         right_layer_txt,
         left_layer_txt,
         reset_output,
         header]

cmd_box = HBox([items[10], items[18]])
left_box = VBox([items[5], items[6], items[12],items[15], items[14], items[0], items[1],items[9],items[9],items[9],cmd_box])
right_box = VBox([items[7], items[8], items[11], items[13], items[9], items[2], items[17], items[3], items[16]])

visualization_widgets = HBox([left_box, right_box, ])

""" Reclassification """
from ipyleaflet import WidgetControl
from ipywidgets import Button, HBox, VBox
from ipywidgets import HBox, VBox
from ipywidgets import Layout
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

placeholder_header = widgets.Label(value="")
lower_than = widgets.Label(value="-")
greater_than = widgets.Label(value="<")
reclass_header = widgets.Label(value="Reclassify Image")
class_header = widgets.Label(value="Number of classes:")
slide_layout= Layout(width='100%') #object_position = '{horz} {vert}'.format(horz=horz, vert=vert))
layername_header = widgets.Label(value="New Layername")

search_layer_txt = widgets.Text(placeholder='Layername',disabled=False,layout = {'width': '75%'})

new_name_txt_input_reclass = widgets.Text(placeholder='Your Layer Title',disabled=False,layout = {'width': '75%'})

interact_slider = interactive(enable_classes,  n = widgets.IntSlider(min=0, max=5, value=0, step=1, layout=Layout(width='300px')))

class_val_layout= Layout(width='95%')

class_val_1 = widgets.FloatText(value=1.5,disabled=True,layout =class_val_layout)

class_val_1_1 = widgets.FloatText(value=2.4,disabled=True,layout =class_val_layout)

class_val_2 = widgets.FloatText(value=2.5,disabled=True,layout =class_val_layout)

class_val_2_1 = widgets.FloatText(value=3.4,disabled=True,layout =class_val_layout)

class_val_3 = widgets.FloatText(value=3.5,disabled=True,layout =class_val_layout)

class_val_3_1 = widgets.FloatText(value=4.4,disabled=True,layout =class_val_layout)

class_val_4 = widgets.FloatText(value=4.5,disabled=True,layout =class_val_layout)

class_val_4_1 = widgets.FloatText(value=5.4,disabled=True,layout =class_val_layout)

class_val_5 = widgets.FloatText(value=5.5,disabled=True,layout =class_val_layout)

class_val_5_1 = widgets.FloatText(value=6.4,disabled=True,layout =class_val_layout)

color_picker_layout= Layout(width='75%')

color_1 = widgets.ColorPicker(concise=True,value='#3b069d',disabled=True,layout=color_picker_layout)

color_2 = widgets.ColorPicker(concise=True,value='#0f5601',disabled=True,layout=color_picker_layout)

color_3 = widgets.ColorPicker(concise=True,value='#0aff0e',disabled=True,layout=color_picker_layout)

color_4 = widgets.ColorPicker(concise=True,value='#f9e401',disabled=True,layout=color_picker_layout)

color_5 = widgets.ColorPicker(concise=True,value='#d32222',disabled=True,layout=color_picker_layout)


classname_info = widgets.Label(value="* 1. class includes values outside the given range")
classname_1 = widgets.Label(value="2.")
classname_2 = widgets.Label(value="3.")
classname_3 = widgets.Label(value="4.")
classname_4 = widgets.Label(value="5.")
classname_5 = widgets.Label(value="6.")

from ipywidgets import GridspecLayout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider



items = [placeholder_header,
            lower_than,
            class_val_1,
            class_val_2,
            class_val_3,
            class_val_4,
            class_val_5,
            color_1,
            color_2,
            color_3,
            color_4,
            color_5,
            greater_than,
            classname_1,
            classname_2,
            classname_3,
            classname_4,
            classname_5,
            class_header,
            reclass_header,
            interact_slider,
            search_layer_txt,
            layername_header,
            new_name_txt_input_reclass,
            class_val_1_1,
            class_val_2_1,
            class_val_3_1,
            class_val_4_1,
            class_val_5_1,
            classname_info] 


box_layout = Layout(width='150px', align_items='flex-start', align_content ='flex-start')
box_layout2 = Layout(width='50px',align_items='center') 

left_box = VBox([items[2],items[3],items[4],items[5],items[6]],layout =box_layout)
right_box_op = VBox([ items[1],  items[1],items[1], items[1],items[1]], layout =box_layout2)
right_box = VBox([items[24], items[25], items[26], items[27], items[28]], layout = box_layout)
right_box_2 = VBox([ items[13], items[14], items[15], items[16], items[17]], layout = box_layout2)
right_box_1 = VBox([ items[7], items[8], items[9], items[10], items[11]], layout = box_layout)

reclass_widgets = HBox([right_box_2,left_box,right_box_op, right_box, right_box_1])

header = VBox([ items[19],items[21], items[0],items[18], items[29] ])
slider = VBox([items[22], items[23],items[0],items[20]])
top_widgets = HBox([header,slider])


#grid = GridspecLayout(9, 5, height='10%', width='20%')
grid = GridspecLayout(4, 1, height='auto')
grid[0:2, :] = top_widgets
grid[2:4, :1] = reclass_widgets


cmd_box = HBox([submit_btn_reclass, reset_output])



grid_widgets = VBox([grid, cmd_box])
grid_widgets



""" Download """


layername_txt = widgets.Text(placeholder='Choose Layer by Name', disabled=False)


download_submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style)

download_img_cb = widgets.Checkbox(
    value=False,
    description='Download single Image (GeoTiff)',
    disabled=False,
    indent=False,
    #**align_kw,
)
download_col_cb = widgets.Checkbox(
    value=False,
    description='Download Image Collection (GeoTiff)',
    disabled=False,
    indent=False,
    #**align_kw,
)

download_feature_cb = widgets.Checkbox(
    value=False,
    description='Download last drawn Feature (Geojson)',
    disabled=False,
    indent=False,
    #**align_kw,
)

download_header = widgets.Label(value="Download Collection, Image or Feature")

items = [download_header,layername_txt, placeholder_header, download_col_cb, download_img_cb, download_feature_cb, download_submit] 

left_box = VBox([items[0],items[1],items[2],items[3],items[2],items[4],items[2],items[5],items[2],items[2],items[6]])
download_widgets = HBox([left_box])

""" Tab Layout """

# TraitError: The 'children' trait of a Tab instance must be a tuple,
# but a value of class 'ipywidgets.widgets.widget_box.VBox' was specified.
# Following which, I passed it as a tuple:
tabs=widgets.Tab((data_widgets, operations_widgets, visualization_widgets, grid_widgets),
                layout=tab_layout)

tabs.set_title(0, 'Data')
tabs.set_title(1, 'Operations')
tabs.set_title(2, 'Vizualisation')
tabs.set_title(3, 'Reclassification')


placeholder_about = widgets.Label(value="Hier folgen Informationen über das Projekt.")
placeholder_tut = widgets.Label(value="Tipps/Tricks/Empfehlungen für die Verwendung der Anwendung")


""" About """

about_information = '<div style="padding-right: 12%; padding-left: 3%;">' + \
'<img src="https://gms.dienste.uni-osnabrueck.de/templates/uni_osnabrueck/images/header_logo.gif" width="250" />' \
+ '<h2>Toolkit for Time Series Analysis with Google Earth Engine</h2>' \
+ '<p style="text-align: justify;">The following toolkit allows to perform individual time series analyses. The focus of the application is on the analysis of air quality based on Sentinel 5P. All data is provided via the Google Earth Engine API. </p>' \
+ '<p style="margin:10px 5px 5px;">Institut for Informatics<br>Studyproject 2021<br>Yannick Reinhardt</p>'+'<a style="text-decoration: underline" href="https://github.com/yreinhardt">View source code on GitHub</a>' \
+ '</div>'


#about_text = HTML(logo_university+test +test1+about_reference)

about_text =HTML(about_information)
about_widgets = HBox([about_text])

""" Tutorial """
#css_str = '<style>.foo{color:#ff0000;} )} </style>'
html_info=HTML('<h2>Tutorial </h2>'+'<p style="text-align: justify;">Lorem ipsum dolor sit amet, consectetuer adipiscing elit,  \
               sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam  \
               erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation  \
               ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat. \
               Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat. \
               Lorem ipsum dolor sit amet, consectetuer adipiscing elit,  \
               sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam  \
               erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation  \
               ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat. \
               Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequa..</p>')


info_widgets = HBox([html_info])


tabs_=widgets.Tab((download_widgets, info_widgets, about_widgets),layout=tab_layout)

tabs_.set_title(0, 'Download')
tabs_.set_title(1, 'Tutorial')
tabs_.set_title(2, 'About')

left = HBox([tabs])
right = HBox([tabs_])

grid = GridspecLayout(1, 5, height='auto')
grid[:, :3] = left
grid[:, 3:] = right
grid


GridspecLayout(children=(HBox(children=(Tab(children=(HBox(children=(VBox(children=(Label(value='Sentinel-5P B…

In [59]:
""" Visualization Paletts"""

no2_vis = {
  'min': 0,
  'max': 0.0002,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

so2_vis = {
  'min': 0.0,
  'max': 0.0005,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

co_vis = {
  'min': 0,
  'max': 0.05,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

ch4_vis = {
    'min': 1750,
    'max': 1900,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

aii_vis = {
    'min': -1,
    'max': 2.0,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}


hcho_vis = {
    'min': 0.0,
    'max': 0.0003,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

o3_vis = {
    'min': 0.12,
    'max': 0.15,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

diff_vis = {
    'min': 0.0,
    'max': 50.0,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

reclass_vis = {
  'min': 1,
  'max': 6,
  'palette': ['#ffffff','#3b069d' ,'#0f5601', '#0aff0e', '#f9e401', '#d32222']
}

In [ ]:
def submit_data(b):
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:

            # get band selection
            gas, product, vis = get_band(band_dd.value)
            
            geom = get_geometry()
            
            # get collection
            collection = get_image_collection(gas, product, vis, geom, start_date_dd.value, end_date_dd.value)
            
            # clip collection
            collection = clip(geom, collection)
            
            if intervall_cb.value == True:
                # get date delta for intervall
                delta = abs(start_date_dd.value - end_date_dd.value)
                delta = delta.days
                delta = delta/intervall_int_input.value
                number_of_images = int(delta)+1
                collection = create_interval(collection, start_date_dd.value, number_of_images, intervall_int_input.value, 'day')

            
            # add new layer with its vis as list to dict
            created_layer[new_name_txt_input.value] = [collection,
                                                       vis,
                                                       geom,
                                                       gas,
                                                       band_dd.value,
                                                       start_date_dd.value,
                                                       end_date_dd.value]
            
            # add layer to map
            Map.addLayer(collection, vis, new_name_txt_input.value)
            
            Map.remove_colorbar()

            add_colorbar(vis)

    
            output_widget.clear_output() 

            
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        else:
            output_widget.clear_output()
            Map.default_style = {'cursor': 'wait'} 
            print("Successfully created new Layer %s" % new_name_txt_input.value)
            Map.default_style = {'cursor': 'default'}
            time.sleep(5) 
            output_widget.clear_output()


In [34]:
clip_feautre_layer_cb.observe(change_usability_clip_cb)
reset_output.on_click(clear_output)
submit_btn_data.on_click(submit_data)


In [35]:
def submit_operations(b):
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            # get band selection
            gas, product, vis = get_band(band_dd.value)
            
            # get input geometry
            geom = get_geometry()
            
            if spatial_stat_cb.value == True:
                # get existing layer
                layer, vis, geom, gas, band, start_date, end_date = get_layer_by_name(stack_search_txt.value)

                # aggregate stack
                layer = aggregate_stack(layer, stat_dd.value)
            
                # add new layer with its vis as list to dict
                created_layer[new_name_txt_input_op.value] = [layer,
                                                               vis,
                                                               geom,
                                                               gas,
                                                               band,
                                                               start_date,
                                                               end_date]
                
                # add layer to map
                Map.addLayer(layer, vis, new_name_txt_input_op.value)
                
                Map.remove_colorbar()

                add_colorbar(vis)
                
            elif  diff_col_cb.value == True:
                layer_diff, diff_viz, geom=  get_difference(first_search_txt.value, second_search_txt.value)
                 # add layer to map
                Map.addLayer(layer_diff, diff_viz, new_name_txt_input_op.value)
                
                Map.remove_colorbar()

                add_colorbar(diff_viz)
                
                created_layer[new_name_txt_input_op.value] = [layer_diff,
                                                               diff_viz,
                                                               geom,
                                                               None,
                                                               None,
                                                               None,
                                                               None]
    
            output_widget.clear_output() 


        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        else:
            output_widget.clear_output()
            Map.default_style = {'cursor': 'wait'} 
            print("Successfully created new Layer %s" % new_name_txt_input_op.value)
            Map.default_style = {'cursor': 'default'}
            time.sleep(5) 
            output_widget.clear_output()
            
         

In [36]:
# anpassen vlt noch doch dass man legende selbst anpasst, oder statt 100 nur 50 anpasst vlt besser

spatial_stat_cb.observe(change_usability_spatialstat)
diff_col_cb.observe(change_usability_difference)
submit_btn_operations.on_click(submit_operations)

In [37]:
def submit_vis(b):
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:

            if compare_timeseries_cb.value == True or timeseries_cb.value == True:
                ax_x, ax_y, data, length = create_timeseries()
                plt.clear()
                fig = plt.figure(axes=[ax_x, ax_y], marks=[data], title=plot_title_txt.value, legend_location='top-right')
                # adjust layout based on timeseries length
                if length <= 200:
                    fig.layout.height = '370px' 
                    fig.layout.width = '700px' 
                elif length >= 200: 
                    fig.layout.height = '450px' 
                    fig.layout.width = '1000px' 
                    
                    
                output_widget.clear_output() 
                
                #Map.remove_colorbar()

                plt.show()
                # x achse, legende außerhalb plot
                # breite höhe des plots
                # figure css https://bqplot.readthedocs.io/en/latest/_generate/bqplot.figure.Figure.htm
            elif splitview_cb.value == True:
                output_widget.clear_output()
                left_layer, right_layer = create_splitview(left_layer_txt.value, right_layer_txt.value)
                # add split to map instance
                Map.split_map(left_layer=left_layer, right_layer=right_layer)

        except Exception as e:
            print(e)
            print('An error occurred during computation.') 

        Map.default_style = {'cursor': 'default'} 

In [38]:
timeseries_cb.observe(change_usability_timeseries)
splitview_cb.observe(change_usability_splitview)
compare_timeseries_cb.observe(change_usability_timeseries)
submit_btn_vis.on_click(submit_vis)


In [13]:
def submit_reclassification(b):
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:

            # value list with all class thresholds
            value_list =[class_val_1.value, class_val_1_1.value,
             class_val_2.value, class_val_2_1.value,
             class_val_3.value, class_val_3_1.value,
             class_val_4.value, class_val_4_1.value,
             class_val_5.value, class_val_5_1.value]

            # slice values based on slider value n
            # multiply 2 cause uper lower threshold (slider number = 3 but 6 thresholds)
            data_list = value_list[:value_n[0]*2]
            
            # reclassify img based on input
            reclass, geom = reclassify(search_layer_txt.value, data_list)
            reclass_img = reclass.clip(geom)
            
            output_widget.clear_output()

            # add new layer with its vis as list to dict
            created_layer[new_name_txt_input_op.value] = [reclass_img,
                                                           reclass_vis,
                                                           None,
                                                           None,
                                                           None,
                                                           None,
                                                           None]

            # add legend aktuell nicht möglich da keine remove function, sonst stapeln die sich
            # add_legend(legend_colors)
            
            Map.addLayer(reclass_img, reclass_vis, new_name_txt_input_reclass.value)

        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        else:
            output_widget.clear_output()
            Map.default_style = {'cursor': 'default'} 
            print ("Successfully created new Layer %s" % new_name_txt_input_reclass.value)
            time.sleep(5) 
            output_widget.clear_output()


In [14]:
submit_btn_reclass.on_click(submit_reclassification)

In [41]:
def submit_download(b):
    
    # widget loader verwenden um dann auch suscessfully created message anzuzeigen
    with output_widget:
        output_widget.clear_output()
        print("Computing...")
        try:

            # use download dir and create folder collection + random string
            download_dir = os.path.join(os.path.expanduser('~'), 'Downloads/Download_GEE_'+ geemap.random_string(3))
            
            # create folder if not exist
            if not os.path.exists(download_dir):
                os.makedirs(download_dir)
            

            
            # export feature collection, single image or feature collection at download dir
            #if download_col_cb.value == True:
                # get layer
                #layer, vis, geom, gas, band, start_date, end_date = get_layer_by_name(layername_txt.value)
                # convert feature collection to geometry object for export
                #geom = geom.geometry()
                #with output_widget_export:
                   # geemap.ee_export_image_collection(layer, out_dir=download_dir, region=geom, scale=1200)
                   # output_widget_export.clear_output()
                
                #import shutil
                #shutil.make_archive("image_collection", 'zip', download_dir)
                #filename = os.path.join(download_dir, 'image_collection.zip')
                
                #geemap.create_download_link(download_dir)
                
                #output_widget.clear_output()
                #print(download_dir)
                #print(filename)
                #print("Available Downloadlink:")
                #display(link)
            
            if download_img_cb.value == True:
                # get layer
                layer, vis, geom, gas, band, start_date, end_date = get_layer_by_name(layername_txt.value)
                # convert feature collection to geometry object for export
                geom = geom.geometry()
                with output_widget_export:
                    filename = os.path.join(download_dir, 'image.tif')
                    geemap.ee_export_image(layer, filename=filename, scale=1200, file_per_band=False, region=geom)
                    output_widget_export.clear_output()
                
                link = geemap.create_download_link(filename, title="Click here to download ")
                
                output_widget.clear_output()
                print("Available Downloadlink:")
                display(link)
            
            elif download_feature_cb.value == True:
                with output_widget_export:
                    feature = get_drawn_feature()
                    filename = os.path.join(download_dir, 'feature.geojson')
                    geemap.ee_export_geojson(feature, filename= filename)
                    output_widget_export.clear_output()
                    
                link = geemap.create_download_link(filename, title="Click here to download ")
            
                output_widget.clear_output()
                print("Available Downloadlink:")
                display(link)
            


            
            
        except OSError:
            print ("Creation of the directory %s failed" % download_dir)
        else:
            Map.default_style = {'cursor': 'wait'}
            #with output_widget:
                #output_widget.clear_output()
                #print ("Successfully downloaded at directory %s" % download_dir)
                #Map.default_style = {'cursor': 'default'} 
                #time.sleep(5) 
                #output_widget.clear_output()
            
        

In [42]:
download_submit.on_click(submit_download)  
download_img_cb.observe(change_download_img)
download_col_cb.observe(change_download_col)
download_feature_cb.observe(change_download_feat)